In [7]:
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd

In [8]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string

def findFiles(path): 
    return glob.glob(path)

def unicodeToAscii(string):
    return ''.join(
        c for c in unicodedata.normalize('NFD', string)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )
# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters) # 57

for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories) # 18

In [9]:
# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

In [14]:
df = pd.read_csv('data/name.csv')
df

,name,country
0,Khoury,Arabic
1,Nahas,Arabic
2,Daher,Arabic
3,Gerges,Arabic
4,Nazari,Arabic
...,...,...
20069,Truong,Vietnamese
20070,Van,Vietnamese
20071,Vinh,Vietnamese
20072,Vuong,Vietnamese


In [26]:
class RnnDataset(Dataset):
    def __init__(self, path):
        from pandas import read_csv
        import numpy as np
        data = read_csv(path)
        name = np.array(data['name'])
        name = torch.tensor(name)
        self.name = name
        ##one-hot encoding
        country = np.array(data['country'])
        country = torch.tensor(country)
        self.country = country

    def __getitem__(self, index):
        
        return (self.name[index], self.country[index])

    def __len__(self):
        return 20074

In [27]:
train_loader = DataLoader(dataset=RnnDataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=None,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None, prefetch_factor=2,
           persistent_workers=False)

In [28]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)

        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        ouput = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [29]:
model = RNN(57, 128, 18) # n_letters=57, hidden_size=128, n_categories=18

loss_fn = nn.NLLLoss()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [34]:
train_iter = iter(train_loader)
for i in range(10):
    print(next(train_iter))

TypeError: object of type 'type' has no len()

In [31]:
for i in range(1, 11):
    for (name, country) in train_loader:
        hidden = model.initHidden()

        for j in range(name.size()[0]):
            output, hidden = model(name[j], hidden)
        
        loss = loss_fn(output, country)

        optimizer.zero_grad()
        loss.backwar()
        optimizer.step()

        print(loss.item())


TypeError: object of type 'type' has no len()